In [27]:
import torch, wandb
import numpy as np
np.set_printoptions(threshold=np.inf, precision=10, linewidth=10**4)
torch.set_printoptions(threshold=np.inf, precision=10, linewidth=10**4)
api = wandb.Api()
print(api)

project_name = 'hippo'
run_id = wandb.run.id  # 現在のランIDを取得
print(run_id)

runs = api.runs(project_name)
print(runs)

for idx, run in enumerate(runs):
    print(idx, run.id)
    print(run)
    metrics = run.history()
    # for rep, met in enumerate(metrics):
    #     print(met)
    
    artifacts = run.logged_artifacts()  # ランに関連する全てのアーティファクト
    for artifact in artifacts:
        print(artifact)
        if 'model' in artifact.name.lower():  # モデルアーティファクトを特定する
            print(f"Artifact Name: {artifact.name}")
            print(f"Artifact Type: {artifact.type}")
            print(f"Artifact Metadata: {artifact.metadata}")  # メタデータにモデルに関する情報が含まれていることがあります
    if idx==2:
        break

# run = api.run("ユーザー名/プロジェクト名/ランID")
# artifact = run.use_artifact("モデル名:latest") #（例：last.ckpt）
# artifact_dir = artifact.download()

# model = MyModel.load_from_checkpoint(artifact_dir + "/last.ckpt") # モデル読み込み（Lightningなど使ってるなら）

x2hzde8t
<Runs jingchuan0guan-the-university-of-tokyo-hospital/hippo>
0 bxdoprfi
<Run jingchuan0guan-the-university-of-tokyo-hospital/hippo/bxdoprfi (finished)>
<Artifact QXJ0aWZhY3Q6MTU4MjYzMzQyOQ==>
1 wxox9i81
<Run jingchuan0guan-the-university-of-tokyo-hospital/hippo/wxox9i81 (finished)>
<Artifact QXJ0aWZhY3Q6MTY0NTIxMTYzMQ==>
2 u4c4hkof
<Run jingchuan0guan-the-university-of-tokyo-hospital/hippo/u4c4hkof (finished)>
<Artifact QXJ0aWZhY3Q6MTY0OTMyOTU1MQ==>


In [ ]:
# @hydra.main(config_path="./configs/model/", config_name="s4d", version_base=None)
# def get_model(model_config: DictConfig):
#     print(model_config)  # DictConfigとしてアクセス可能
#     for key in model_config.keys():
#         print(key, model_config[key])
#     model = utils.instantiate(registry.model, model_config)#.load_from_checkpoint("path/to/model.ckpt")
#     print(model)
    
#     state_dict = model.state_dict()
#     for k, v in state_dict.items():
#         print(k, v.shape)

# import sys
# sys.path.append('/home/kan/ML_application/s4/src')
# layer_key = ["s4d", ][0]
# print(registry.model["model"])
# print(registry.layer[layer_key])
# print(registry.model["model"](layer=registry.layer[layer_key]))
# ckpt ファイルをロード
# model = utils.instantiate(registry.model, layer="s4d", )
# registry.model["model"](layer=layer).load_from_checkpoint("path/to/model.ckpt")

# for root, dirs, files in os.walk(base_dir):
#     # 今の階層の深さを数える
#     depth = root[len(base_dir):].count(os.sep)
#     print(depth)
#     if depth > max_depth:
#         # さらに深い階層の探索をやめる
#         dirs[:] = []
#         continue

#     for file in files:
#         print(os.path.join(root, file))

In [ ]:
from pytorch_lightning import LightningModule
# import src.utils as utils
from pathlib import Path

import hydra
from hydra import initialize, compose
from omegaconf import DictConfig, OmegaConf
# from hydra.utils import instantiate

from src.utils import registry, instantiate

def check_model(model_name="model", cfg_name="config", cfg_path="./configs/model/",  model_path="", ):
    with initialize(config_path=cfg_path):
        cfg = compose(config_name=cfg_name)
    print("cfg", cfg)
    OmegaConf.set_struct(cfg, False)
    # cfg.model._target_ = "torch.nn.Identity" 
    ModelClass = hydra.utils.get_class(registry.model[model_name])
    # model = ModelClass(cfg)
    
    # model = instantiate(cfg.model) # registry.model,
    model = instantiate(registry.model, cfg) # モデル構造のインスタンス化（この時点でランダム初期化）
    print(model)
    print("type", type(model))
    ckpt = torch.load(model_path, map_location="cpu")# チェックポイントからstate_dictを読み込み
    # state_dict = {k.replace("model.", ""): v for k, v in ckpt["state_dict"].items()}
    # model.load_state_dict(state_dict)
    print("Checkpoint keys:", ckpt.keys())  # 追加
    
    state_dict = ckpt.get("state_dict", ckpt)
    model.load_state_dict(state_dict)
    # if "state_dict" in ckpt:
    #     state_dict = ckpt["state_dict"]
    # else:
    #     state_dict = ckpt
    # model.load_state_dict(state_dict)
    
    # for key in cfg.keys():
    #     print(key, cfg[key])
    # model = utils.instantiate(registry.model, cfg).load_from_checkpoint(model_path)

    print(model)
    
# /home/kan/ML_application/s4/outputs/2025-04-21/11-19-18/checkpoints/

base_dir = "/home/kan/ML_application/s4/outputs/"
max_depth = 2

base_dir_ = Path(base_dir)
for file in base_dir_.glob("*/*/checkpoints/*"):
    if file.is_file():
        parent_path = file.parents[1]
        print(parent_path)
        print(file)
        relative_path = parent_path.parts[parent_path.parts.index("outputs"):]  # s4 以降の部分を抽出
        cfg_path = "/".join(relative_path)+ "/.hydra"
        print(cfg_path)
        check_model(model_name="model", cfg_path=cfg_path, model_path=file)


/home/kan/ML_application/s4/outputs/2025-04-20/22-59-56
/home/kan/ML_application/s4/outputs/2025-04-20/22-59-56/checkpoints/last.ckpt
outputs/2025-04-20/22-59-56/.hydra
cfg {'train': {'seed': 0, 'interval': 'epoch', 'monitor': 'val/accuracy', 'mode': 'max', 'ema': 0.0, 'test': False, 'debug': False, 'ignore_warnings': False, 'state': {'mode': None, 'chunk_len': None, 'overlap_len': None, 'n_context': 0, 'n_context_eval': '${.n_context}'}, 'sweep': None, 'group': None, 'benchmark_step': False, 'benchmark_step_k': 1, 'benchmark_step_T': 1, 'checkpoint_path': None, 'visualizer': 'filters', 'disable_dataset': False}, 'wandb': {'project': 'hippo', 'group': '', 'job_type': 'training', 'mode': 'online', 'save_dir': None, 'id': None}, 'trainer': {'accelerator': 'cuda', 'devices': [0, 1, 2, 3, 4, 5, 6, 7], 'accumulate_grad_batches': 1, 'max_epochs': 100, 'gradient_clip_val': 0.0, 'log_every_n_steps': 10, 'limit_train_batches': 1.0, 'limit_val_batches': 1.0, 'enable_progress_bar': True}, 'loader

/tmp/ipykernel_649803/159137829.py:14: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path=cfg_path):


InstantiationException: Cannot instantiate config of type type.
Top level config must be an OmegaConf DictConfig/ListConfig object,
a plain dict/list, or a Structured Config class or instance.